# Load Model Weights

In [1]:
%time
import os
import numpy as np

from ct_sam.utils.io_utils import load_module_from_file
from ct_sam.builder import build_sam
from ct_sam.predictor import SamPredictor

import torch
print(torch.__version__)


checkpoint = "/root/CCVL/ct-sam3d/ct_sam/models/params.pth"

config_file = os.path.join(os.path.dirname(checkpoint), "config.py")
assert os.path.isfile(config_file), "file config.py not found!"
cfg_module = load_module_from_file(config_file)
cfg = cfg_module.cfg

cfg.update({"checkpoint": checkpoint})
sam = build_sam(cfg)
if torch.cuda.is_available():
    sam.cuda()
predictor = SamPredictor(sam, cfg.dataset)
print("predictor device: ", predictor.device)


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs
2.4.1+cu121


You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


predictor device:  cuda:0


# Full Image and Mask View

In [4]:
"""postprocessed data
    1. /root/autodl-tmp/AbdomenAtlasDemoPredictDAPpostprocessed
    2. /root/autodl-tmp/AbdomenAtlasDemoPredictSourceDAPpostprocessed
"""
import SimpleITK as sitk
import itkwidgets
from itkwidgets import view
from tqdm import tqdm
from scipy import ndimage

from ct_sam.utils.frame import voxel_to_world
from ct_sam.utils.resample import flip_itkimage_torai, resample_itkimage_torai, crop_roi_with_center, resample_itkimage_withspacing


CT_INPUT_DIR = "/root/autodl-tmp/VISTA-AbdomenAtlasDemo"
PSEUDO_INPUT_DIR = "/root/autodl-tmp/AbdomenAtlasDemoPredictDAPpostprocessed"
OUTPUT_DIR = "/root/autodl-tmp/AbdomenAtlasDemoPredictCTSAM3D"

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

ALL_TARGET =[f"L{idx}" for idx in range(5, 0, -1)] + \
             [f"T{idx}" for idx in range(12, 0, -1)] + \
            [f"C{idx}" for idx in range(7, 0, -1)]  # L-5 + T-12 + C-7

target_labels = [i for i in range(66, 42, -1)]
num_samples = 1


def compute_dsc_np(predict, targets, threshold=0.0, smooth=1):
    intersection = (predict * targets).sum()
    dice = (2.0 * intersection + smooth) / (predict.sum() + targets.sum() + smooth)
    return dice


# Crop and Patch Interaction

In [5]:



volumes = ["BDMAP_000000" + num for num in ["01", "05", "06", "10", "12", "19", "22", "26", "31"]] 
# volume = "BDMAP_00000012"
for volume in volumes:

    if not os.path.exists(os.path.join(OUTPUT_DIR, f"{volume}")):
        os.mkdir(os.path.join(OUTPUT_DIR, f"{volume}"))
        os.mkdir(os.path.join(OUTPUT_DIR, f"{volume}", "segmentations"))

    image_raw = sitk.ReadImage(f"{CT_INPUT_DIR}/{volume}/ct.nii.gz")   # H W C
    image = resample_itkimage_torai(image_raw, [1.5, 1.5, 1.5], "linear", -1024)    # C H W

    mask_gt = None
    try:
        mask_gt_raw = sitk.ReadImage(f"{PSEUDO_INPUT_DIR}/{volume}.nii.gz") # H W C
        mask_gt = resample_itkimage_torai(mask_gt_raw, [1.5, 1.5, 1.5], interpolator="nearest", pad_value=0)    # C H W

    except Exception as e:
        print(f"read mask failed: {e}")


    mask_pred_list = list()
    empty_class = list()
    pbar = tqdm(enumerate(target_labels), total=len(target_labels), desc=f"Predicting {volume}")
    for idx, target_label in pbar:
        # Prepare an empty volume to store predictions
        mask_pred = np.zeros_like(sitk.GetArrayFromImage(mask_gt))  # c h w
        mask_pred = sitk.GetImageFromArray(mask_pred)   # w h c
        mask_pred.CopyInformation(mask_gt)

        # Preprocess: sample point prompt and crop a roi for input
        mask_gt_coord = np.argwhere(sitk.GetArrayFromImage(mask_gt).transpose((2, 1, 0)) == target_label)   # c h w
        try:
            assert len(mask_gt_coord) > 0
            # random_point = mask_gt_coord[np.random.choice(len(mask_gt_coord))]
            random_point = np.rint(mask_gt_coord.mean(0)).astype(np.int32)
            # random_point = (mask_gt_coord.max(0) + mask_gt_coord.min(0)) // 2
        except AssertionError:
            empty_class.append(target_label)
            mask_pred_list.append(mask_pred)
            pbar.set_postfix(dict(empty_class=empty_class))
            continue
        center_v = random_point
        center_w = voxel_to_world(image, center_v)
        x_axis, y_axis, z_axis = np.array(image.GetDirection()).reshape(3, 3).transpose()
        image_patch = crop_roi_with_center(image, center_w, image.GetSpacing(), x_axis, y_axis, z_axis, [64, 64, 64], "linear", -1024)

        if mask_gt is not None:
            mask_gt_patch = crop_roi_with_center(mask_gt, center_w, mask_gt.GetSpacing(), x_axis, y_axis, z_axis, [64, 64, 64], "nearest", 0)
            mask_gt_patch = sitk.GetArrayFromImage(mask_gt_patch).astype(np.int32).transpose((2, 1, 0))  # c h w

        # Inference: set the image and make predictions
        predictor.set_image(image_patch)

        # sample point prompt(s) from interior mask
        distance = ndimage.distance_transform_edt(mask_gt_patch == target_label)    # Step 1: Compute the distance transform of the mask
        threshold = 0.5 * distance.max()    # Step 2: Define a threshold to exclude surface points
        interior_mask = distance >= threshold   # Step 3: Create a mask of interior points based on the distance threshold
        interior_coords = np.argwhere(interior_mask)    # Step 4: Get the coordinates of the interior points
        points = interior_coords[np.random.choice(len(interior_coords), size=num_samples)]
        labels = np.array([1] * num_samples)

        mask_input = None

        mask, scores, logits = predictor.predict(
            point_coords=points,
            point_labels=labels,
            multimask_output=False,
            mask_input=mask_input,
        )

        # Postprocess: reset the image and save nifti
        min_p = random_point - 32
        min_m = 0 - min_p
        min_m[min_m < 0] = 0
        min_p[min_p < 0] = 0

        max_p = random_point + 32
        max_m = max_p - mask_gt.GetSize()
        max_m[max_m < 0] = 0
        max_m = 64 - max_m
        
        mask_pred[min_p[0]:max_p[0], min_p[1]:max_p[1], min_p[2]:max_p[2]] = (idx+1) * \
            mask[min_m[0]:max_m[0], min_m[1]:max_m[1], min_m[2]:max_m[2]]

        mask_pred_list.append(mask_pred)
        # pbar.update()

    # saving results for each class
    for idx in tqdm(range(24), desc="Saving"):
        mask_pred = sitk.Resample(mask_pred_list[idx], mask_gt_raw, sitk.Transform(), sitk.sitkNearestNeighbor, 0.0, mask_pred_list[idx].GetPixelID())
        sitk.WriteImage(mask_pred, os.path.join(OUTPUT_DIR, f"{volume}", f"segmentations", f"vertebrae_{ALL_TARGET[idx]}.nii.gz"))

    # saving combined labels (WARNING: risk of overlap predictions)
    mask_pred = sum(mask_pred_list)
    mask_pred = sitk.Resample(mask_pred, mask_gt_raw, sitk.Transform(), sitk.sitkNearestNeighbor, 0.0, mask_pred.GetPixelID())
    sitk.WriteImage(mask_pred, os.path.join(OUTPUT_DIR, f"{volume}", f"combined_labels.nii.gz"))

Saving: 100%|██████████| 24/24 [00:22<00:00,  1.07it/s]
